In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [2]:
# Creating function to convert odds to probability
def calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100

# Creating team map to align w/ final output
team_map = {
    'Wizards' : 'Washington Wizards',
    'Celtics' : 'Boston Celtics',
    'Pelicans' : 'New Orleans Pelicans',
    'Knicks' : 'New York Knicks',
    'Pistons' : 'Detroit Pistons',
    'Magic' : 'Orlando Magic',
    '76ers' : 'Philadelphia 76ers',
    'Hawks' : 'Atlanta Hawks',
    'Pacers' : 'Indiana Pacers',
    'Raptors' : 'Toronto Raptors',
    'Grizzlies' : 'Memphis Grizzlies',
    'Heat' : 'Miami Heat',
    'Bulls' : 'Chicago Bulls',
    'Jazz' : 'Utah Jazz',
    'Bucks' : 'Milwaukee Bucks',
    'Spurs' : 'San Antonio Spurs',
    'Warriors' : 'Golden State Warriors',
    'Thunder' : 'Oklahoma City Thunder',
    'Timberwolves' : 'Minnesota Timberwolves',
    'Nuggets' : 'Denver Nuggets',
    'Suns' : 'Phoenix Suns',
    'Cavaliers' : 'Cleveland Cavaliers',
    'Mavericks' : 'Dallas Mavericks',
    'Kings' : 'Sacramento Kings',
    'Hornets' : 'Charlotte Hornets',
    'Trail Blazers' : 'Portland Trailblazers',
    'Nets' : 'Brooklyn Nets',
    'Lakers' : 'Los Angeles Lakers',
    'Clippers' : 'Los Angeles Clippers',
    'Rockets' : 'Houston Rockets'
}

# Instantiating WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/nba/odds')

# Navigating to ML
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')

# Getting table from HTML
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


IndexError: list index out of range

In [8]:
odds.dropna(inplace = True)
odds.reset_index(drop = True, inplace = True)
odds

,Scheduled,Open,Best Odds,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,SpursSAS521CavaliersCLE522,+215-260,+198ML-210ML,+185ML-225ML,+175ML-210ML,+198ML-240ML,+185ML-225ML,N/AN/A,+185ML-225ML,+185ML-225ML,+175ML-210ML,+190ML-230ML,+175ML-210ML
1,BullsCHI523HornetsCHA524,-115-104,-121ML+108ML,-125ML+105ML,-121ML+102ML,-126ML+108ML,-125ML+105ML,N/AN/A,-125ML+105ML,-125ML+105ML,-121ML+102ML,-125ML+105ML,-121ML+102ML
2,RaptorsTOR525ThunderOKC526,-480+370,-450ML+400ML,-510ML+375ML,-530ML+390ML,-500ML+385ML,-500ML+375ML,N/AN/A,-450ML+340ML,-500ML+375ML,-530ML+390ML,-525ML+400ML,-530ML+390ML
3,TimberwolvesMIN527KingsSAC528,-405+320,-400ML+340ML,-410ML+310ML,-435ML+335ML,-430ML+340ML,-400ML+310ML,N/AN/A,-450ML+340ML,-400ML+310ML,-435ML+335ML,-425ML+325ML,-435ML+335ML
4,WarriorsGSW531JazzUTA532,+140-177,+115ML-125ML,+105ML-125ML,+110ML-132ML,+110ML-130ML,+115ML-135ML,N/AN/A,+115ML-140ML,+115ML-135ML,+110ML-132ML,+110ML-130ML,+110ML-132ML
5,LakersLAL529Trail BlazersPOR530,-230+190,-420ML+350ML,-435ML+330ML,-435ML+335ML,-420ML+330ML,-425ML+330ML,N/AN/A,-450ML+350ML,-425ML+330ML,-435ML+335ML,-425ML+325ML,-435ML+335ML


In [9]:
odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():
    
    # Retreiving teams
    teams = {}
    for key in team_map.keys():
        if row.Scheduled.find(key) != -1:
            teams[row.Scheduled.find(key)] = key
    keys = []
    for key in teams.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams[keys[0]]
        away_team = teams[keys[1]]
    else:
        home_team = teams[keys[1]]
        away_team = teams[keys[0]]

    # Retreiving odds
    ml_string = row['Unnamed: 5']
    if len(ml_string) == 12:
        ml_string = ml_string.replace('ML', '')
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    if len(ml_string) == 13:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])

# Adding a message if games are passed over for same reason
if len(odds_df) != len(odds):
    print("At least one game not included in odds df")

In [10]:
odds_df

,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob,Away_Prob
0,Cleveland Cavaliers,San Antonio Spurs,-240.0,198.0,70.588235,33.557047
1,Charlotte Hornets,Chicago Bulls,108.0,-126.0,48.076923,55.752212
2,Oklahoma City Thunder,Toronto Raptors,385.0,-500.0,20.618557,83.333333
3,Sacramento Kings,Minnesota Timberwolves,340.0,-430.0,22.727273,81.132075
4,Utah Jazz,Golden State Warriors,-130.0,110.0,56.521739,47.619048
5,Portland Trailblazers,Los Angeles Lakers,330.0,-420.0,23.255814,80.769231
